In [1]:
import github
from secrets import MY_TOKEN

g = github.Github(login_or_token=MY_TOKEN, per_page=100)

In [2]:
with open('selected1.txt', 'r') as f:
    full_names = f.readlines()

full_names = list(map(lambda x: x.strip(), full_names))
full_names = list(dict.fromkeys(full_names))
full_names

['matomo-org/matomo',
 'rubinius/rubinius',
 'ruby-git/ruby-git',
 'spree/spree',
 'thoughtbot/paperclip',
 'rails/rails',
 'cucumber/cucumber-ruby',
 'sparklemotion/mechanize',
 'vubiostat/r-yaml',
 'activemerchant/active_merchant',
 'sparklemotion/nokogiri',
 'kennethkalmer/powerdns-on-rails',
 'Floppy/climatestats',
 'briandfoy/test-file',
 'xdev/blackbird',
 'sameeratgmail/sameeratgmail',
 'zencoder/rvideo',
 'sameeratgmail/sample-project2',
 'thoughtbot/clearance',
 'beastaugh/tarski',
 'Shopify/shopify_app',
 'Floppy/carbon-diet',
 'jekyll/jekyll',
 'mojolicious/mojo',
 'publify/publify',
 'internetarchive/openlibrary',
 'internetarchive/bookreader',
 'wraith/wraith',
 'showdownjs/showdown',
 'rnhurt/gradesheet',
 'halostatue/color',
 'mime-types/ruby-mime-types',
 'nlamirault/cl-lastfm',
 'justingit/dada-mail',
 'bellroy/lesswrong',
 'nlamirault/www',
 'nlamirault/ernestine',
 'rurban/b-generate',
 'sunny/so-nice',
 'nlamirault/cl-gdata',
 'faker-ruby/faker',
 'rgrove/sanitize',

In [3]:
def is_s_label(label_name):
    return 'security' in label_name.lower()

In [4]:
filtered = []
for name in full_names:
    repo = g.get_repo(name)
    sec_count = 0
    for label in repo.get_labels():
        if is_s_label(label.name):
            sec_count += repo.get_issues(labels=[label]).totalCount
    if (sec_count > 0):
        filtered.append(name)

In [5]:
filtered

['matomo-org/matomo',
 'rails/rails',
 'sparklemotion/nokogiri',
 'kennethkalmer/powerdns-on-rails',
 'thoughtbot/clearance',
 'Shopify/shopify_app',
 'jekyll/jekyll',
 'wraith/wraith',
 'showdownjs/showdown',
 'rnhurt/gradesheet',
 'bellroy/lesswrong',
 'chef/chef',
 'paramiko/paramiko',
 'akka/akka',
 'heroku/legacy-cli',
 'factor/factor',
 'textpattern/textpattern',
 'klauern/groovyjersey',
 'freenet/fred',
 'gmacario/gmacario.github.io',
 'nbudin/vellum',
 'jonphipps/Metadata-Registry',
 'apache/couchdb',
 'ThinkUpLLC/ThinkUp',
 'net-ssh/net-ssh',
 'thoughtbot/upcase',
 'vim-syntastic/syntastic',
 'opencats/OpenCATS',
 'orbeon/orbeon-forms',
 'zeromq/libzmq',
 'nette/nette',
 'BitLucid/ninjawars',
 'Arachnid/netboot.me',
 'YaleSTC/shifts',
 'ohmyzsh/ohmyzsh',
 'bossmc/joker',
 'npm/npm',
 'chusopr/casimir',
 'ChicagoBoss/ChicagoBoss',
 'pierrel/SomethingToWear',
 'halfbyte/jan.krutisch.de',
 'joestump/python-oauth2',
 'apache/trafficserver',
 'walkscore/City-Go-Round',
 'clojars/cl

In [6]:
def prepare_to_csv(text):
    return '"' + text.replace('"', '""') + '"' if text else ""

In [7]:
with open(f'data/security_issues.csv', 'w',  encoding='utf-8') as f:
    f.write('repository,number,title,description,comments,is_pr,labels\n')

    for name in filtered:
        repo = g.get_repo(name)
        sec_issues = set()
        for label in repo.get_labels():
            if is_s_label(label.name):
                sec_issues.update([issue for issue in repo.get_issues(labels=[label])])

        for issue in sec_issues:
            f.write(repo.full_name + ',')
            f.write(str(issue.id) + ',')
            f.write(prepare_to_csv(issue.title) + ',')
            f.write(prepare_to_csv(issue.body) + ',')

            comments = issue.get_comments()
            comments_str = ""
            for c in comments:
                comments_str += c.body
            f.write(prepare_to_csv(comments_str) + ',')
            
            if issue.pull_request:
                f.write('yes,')
            else:
                f.write('no,')
            
            labels = ""
            for label in issue.labels:
                labels += label.name + ','

            f.write(prepare_to_csv(labels))
            
            f.write('\n')


In [8]:
import pandas as pd

data = pd.read_csv("data/security_issues.csv")
data

,repository,number,title,description,comments,is_pr,labels
0,matomo-org/matomo,124434256,Preventing timing attacks on authentication,### Background\r\n\r\nSee http://codahale.com/...,"Having thought about this issue a bit more, **...",no,"c: Security,"
1,matomo-org/matomo,104426717,do not allow to set access to non existing web...,"As a Super User, currently I can set user acce...",NaN,no,"c: Security,"
2,matomo-org/matomo,452436369,rate limit scheduled email reports,Email reports in Matomo can be abused to send ...,Maybe an even better (even though complexer to...,no,"Major,c: Security,"
3,matomo-org/matomo,1207536963,`POST` to `matomo.php` Endpoint Returns 204 Wh...,According to the documentation [in the Matomo ...,"@Zozman thanks for the bug report, I think tha...",no,"c: Security,Regression,"
4,matomo-org/matomo,333223420,Encourage strong passwords by indicating when ...,I think it'd be helpful for the admin to have ...,That could also be added in admin when changin...,no,"Enhancement,c: Security,c: Usability,"
...,...,...,...,...,...,...,...
208,spreadthesource/wooki,117995,Enable 'remember me' service,NaN,Postponed to 0.3 for priority reasons\r\n,no,"Security,"
209,spreadthesource/wooki,118406,Enable OpenID/OAuth login,This is common feature for all social websites...,NaN,no,"Security,"
210,spreadthesource/wooki,88579,Password recovery,We should not forget to implement this feature...,NaN,no,"Security,"
211,spreadthesource/wooki,88267,Terms of service,"When creating an account somewhere, there is a...",Ok.\r\nMove to 1.0 since we will provide a too...,no,"Security,"
